In [1]:
import matplotlib.pyplot as plt
import torch
from Net import Net
from Data import Data
from Train import train_free_energy, fit, train_free_energy_two_inputs
import numpy as np
from scipy import integrate
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
torch.manual_seed(3)

%matplotlib qt

#### Make some data

In [2]:
lower = 0
upper = 2*np.pi
data_set = Data(256, lower, upper)
x, y = data_set.get()
matrix = data_set.matrix()
input_var = data_set.input_variables()

#### Implement neural network, optimizer and scheduler

#### Learning rate decay

In [6]:
Layers = [2, 16, 16, 16, 16, 16, 16, 1]
net = Net(Layers)
learning_rate = 0.01
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)

#### An example

In [ ]:
rho = 3.5
lambd = 25
net = train_free_energy_two_inputs(net, x, input_var, rho, lambd, scheduler,
                                   optimizer, matrix, epochs=10000, diagram=True)

#### Pre-train can make the cost curve more obvious

In [5]:
net_dict_prev = {}
rho_list = np.linspace(4, 6, 15)
for i in range (len(rho_list)):
    rho = rho_list[i]
    net = Net(Layers)
    learning_rate = 0.015
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)
    lambd = 15
    net_dict_prev["rho=" + str(i)] = train_free_energy_two_inputs(net, x,  input_var, rho, lambd, scheduler,
                                       optimizer, matrix, epochs=500, diagram=True)

KeyboardInterrupt: 

In [14]:
rho_list = np.linspace(4, 6, 15)
net_dict = {}
output_list = {}
i = 0
for rho in rho_list:
    net = net_dict_prev["rho=" + str(i)]
    learning_rate = 0.02
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)
    lambd = 15
    net_dict["rho=" + str(i)] = train_free_energy_two_inputs(net, x,  input_var, rho, lambd, scheduler,
                                       optimizer, matrix, epochs=2000, diagram=False)
    output_list["rho=" + str(i)] = net_dict["rho=" + str(i)](input_var).data.numpy()
    i+= 1

In [11]:
#这太蠢了
height = np.concatenate([output_list["rho=" + str(0)],
                         output_list["rho=" + str(1)],
                         output_list["rho=" + str(2)],
                         output_list["rho=" + str(3)],
                         output_list["rho=" + str(4)],
                         output_list["rho=" + str(5)],
                         output_list["rho=" + str(6)],
                         output_list["rho=" + str(7)],
                         output_list["rho=" + str(8)],
                         output_list["rho=" + str(9)],
                         output_list["rho=" + str(10)],
                         output_list["rho=" + str(11)],
                         output_list["rho=" + str(12)],
                         output_list["rho=" + str(13)],
                         output_list["rho=" + str(14)]],
                        axis= 1 )

In [12]:
fig3d = plt.figure()
ax = Axes3D(fig3d)
X = np.linspace(0.000001, 2 * np.pi, 256)
rho_list, X = np.meshgrid(rho_list, X)    # x-y grid
ax.plot_surface(rho_list, X , height, rstride=1, cstride=1, cmap=plt.get_cmap('rainbow'))

In [18]:
print (rho_list[1:13])

[[4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 ...
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]
 [4.         4.14285714 4.28571429 ... 5.71428571 5.85714286 6.        ]]
